In [14]:
import numpy as np
from scipy.optimize import least_squares
from itertools import combinations, product

# 数据重新识别
data = {
    "A": {"coords": (110.241, 27.204, 824), "times": [100.767, 164.229, 214.850, 270.065]},
    "B": {"coords": (110.783, 27.456, 727), "times": [92.453, 112.220, 169.362, 196.583]},
    "C": {"coords": (110.762, 27.785, 742), "times": [75.560, 110.696, 156.936, 188.020]},
    "D": {"coords": (110.251, 28.025, 850), "times": [94.653, 141.409, 196.517, 258.985]},
    "E": {"coords": (110.524, 27.617, 786), "times": [78.600, 86.216, 118.443, 126.669]},
    "F": {"coords": (110.467, 28.081, 678), "times": [67.274, 166.270, 175.482, 266.871]},
    "G": {"coords": (110.047, 27.521, 575), "times": [103.738, 163.024, 206.789, 210.306]}
}

# 提取所有设备的ID
device_ids = list(data.keys())

# 定义函数将经纬度转换为长度
def lat_lon_to_length(x, y, x_ref, y_ref):
    x1 = abs(x - x_ref)
    x2 = 360 - x1
    x3 = min(x1, x2)
    dx = 97.304 * x3 * 1000  # 转换为米
    dy = 111.263 * (y - y_ref) * 1000  # 转换为米
    return dx, dy

# 定义函数用于计算误差
def residuals(params, v, t_i, coords):
    x, y, z, t0 = params
    residuals = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x, y, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z - coords[i][2])**2)
        t_calc = t0 + d_i / v
        residuals.append(t_i[i] - t_calc)
    return residuals

# 音速
v = 340.0  # m/s

# 定义主函数，用于计算最佳组合
def find_best_combination():
    all_results = []

    for selected_devices in combinations(device_ids, 6):                           # 修改组数改这里！！！
        # print(f"正在处理设备组合: {selected_devices}")  # 调试信息

        # 生成这些设备的时间组合
        time_combinations = list(product(*[data[device]["times"] for device in selected_devices]))

        # 提取这些设备的坐标和时间
        coords = np.array([data[device]["coords"] for device in selected_devices])

        # 初始化误差列表
        results = []

        # 对每种方案进行计算
        for times in time_combinations:
            t_i = np.array(times)
            initial_guess = np.array([np.mean(coords[:, 0]), np.mean(coords[:, 1]), np.mean(coords[:, 2]), np.min(t_i)])
            result = least_squares(residuals, initial_guess, bounds=([-180, -90, 0, -np.inf], [180, 90, np.inf, np.inf]), args=(v, t_i, coords))
            x_sol, y_sol, z_sol, t0_sol = result.x
            predicted_times = []
            for i in range(len(t_i)):
                dx, dy = lat_lon_to_length(x_sol, y_sol, coords[i][0], coords[i][1])
                d_i = np.sqrt(dx**2 + dy**2 + (z_sol - coords[i][2])**2)
                t_calc = t0_sol + d_i / v
                predicted_times.append(t_calc)
            errors = t_i - np.array(predicted_times)
            total_error = np.sum(errors ** 2)
            results.append((times, (x_sol, y_sol, z_sol), t0_sol, total_error, selected_devices))

        # 按误差排序
        results.sort(key=lambda x: x[3])

        # DFS + 剪枝寻找最佳组合
        def find_combination_with_dfs():
            best_combination = []
            best_error_sum = float('inf')
            
            def dfs(path, times_path, errors_path, used_times, idx):
                nonlocal best_combination, best_error_sum
                
                if len(path) == 4:
                    error_sum = sum(errors_path)
                    if error_sum < best_error_sum:
                        best_error_sum = error_sum
                        best_combination = list(path)
                    return
                
                if idx >= len(results):
                    return
                
                for i in range(idx, len(results)):
                    times, coords_sol, t, err, devices = results[i]
                    # 检查当前组合的时间是否在已使用的时间列表中
                    if any(time in used_times for time in times):
                        continue
                    # 检查预测时间差是否大于5秒
                    if path and (max(times_path + [t]) - min(times_path + [t]) > 5):
                        continue
                    # 检查当前路径的误差和是否已经超过当前最佳误差和
                    if sum(errors_path) + err >= best_error_sum:
                        continue
                    dfs(path + [results[i]], times_path + [t], errors_path + [err], used_times + list(times), i + 1)
            
            dfs([], [], [], [], 0)
            
            return best_combination, best_error_sum
        
        best_combination, best_error_sum = find_combination_with_dfs()
        
        # 将每个设备组合的最优结果添加到总结果中
        all_results.append((best_combination, best_error_sum))

    # 按误差排序
    all_results.sort(key=lambda x: x[1])

    # 输出误差平方之和前四小的方案
    print("误差平方之和前四小的方案:")
    print()
    for i, (comb, error_sum) in enumerate(all_results[:4]):
        print(f"方案 {i + 1}:")
        devices_set = set()
        for j, (times, coords_sol, t, err, devices) in enumerate(comb):
            devices_set.update(devices)
        comb_str = ", ".join(devices_set)
        print(f"  设备组合: {comb_str}")
        for j, (times, coords_sol, t, err, devices) in enumerate(comb):
            print(f"  Times = {times}\n  Predicted coords = {coords_sol}\n  Predicted Time = {t:.3f}, Error Sum = {err:.3f}")
        print(f"  最小误差平方之和: {error_sum:.3f}\n")

# 运行主函数
find_best_combination()

误差平方之和前四小的方案:

方案 1:
  设备组合: A, B, C, G, F, D
  Times = (214.85, 92.453, 75.56, 196.517, 175.482, 210.306)
  Predicted coords = (110.69999774847652, 27.65000003259501, 13467.880482082432)
  Predicted Time = 15.000, Error Sum = 0.000
  Times = (100.767, 112.22, 188.02, 258.985, 266.871, 163.024)
  Predicted coords = (110.4999998115281, 27.310002622795157, 12511.80637527718)
  Predicted Time = 12.002, Error Sum = 0.000
  Times = (270.065, 196.583, 110.696, 94.653, 67.274, 206.789)
  Predicted coords = (110.4999981668525, 27.949996587297814, 11528.453090258321)
  Predicted Time = 13.002, Error Sum = 0.000
  Times = (164.229, 169.362, 156.936, 141.409, 166.27, 103.738)
  Predicted coords = (110.30000054327924, 27.650000023614563, 11476.66325284947)
  Predicted Time = 14.001, Error Sum = 0.000
  最小误差平方之和: 0.000

方案 2:
  设备组合: A, B, C, G, E, D
  Times = (100.767, 112.22, 188.02, 258.985, 118.443, 163.024)
  Predicted coords = (110.49596346797807, 27.318872201503222, 2.609034854599636e-16)
  